In [2]:
import astropy.coordinates as coord
import numpy as np
from gala.units import UnitSystem
from yellowcard.model import TimingArgumentModel 
from yellowcard.coordinates import fiducial_m31_c
import astropy.units as u
from scipy.optimize import minimize
import emcee
import arviz as az

import matplotlib.pyplot as plt


In [3]:
plt.rcParams.update({'font.size':22,'text.usetex':False})

In [5]:
model = TimingArgumentModel(distance = fiducial_m31_c.distance,
                            pm = u.Quantity([fiducial_m31_c.pm_ra_cosdec, fiducial_m31_c.pm_dec]),
                            radial_velocity = fiducial_m31_c.radial_velocity,
                            tperi = 13.7*u.Gyr,
                            distance_err = 10*u.kpc,
                            pm_err = u.Quantity([10*u.microarcsecond/u.yr, 10*u.microarcsecond/u.yr]),
                            radial_velocity_err = 10 * u.km / u.s,
                            tperi_err = .25 * u.Gyr,
                            )

In [6]:
galcen_m31     = fiducial_m31_c.transform_to(model.galcen_frame)
galcen_m31_pos = galcen_m31.data.without_differentials()
galcen_m31_vel = galcen_m31.velocity
galcen_m31_L   = galcen_m31_pos.cross(galcen_m31_vel)
galcen_m31_L   = galcen_m31_L / galcen_m31_L.norm()

In [7]:
e_init   = 0.9
eta_init = 5*u.rad
init_par = {}

init_par['a'] = fiducial_m31_c.distance.value/(1-e_init*np.cos(eta_init))
init_par['ecoseta'] = e_init*np.cos(eta_init)
init_par['esineta'] = e_init*np.sin(eta_init)
init_par['M'] = (4e12*u.Msun).decompose(model.unit_system).value
init_par['Lhatlg'] = galcen_m31_L.xyz

In [ ]:
init_par

In [8]:
result = minimize( lambda *args: -model(*args), model.pack_pars(init_par), method='Powell')

/Users/katiechambe/Documents/Research/projects/yellowCard/yellowcard/keplerianPlane.py:61: RuntimeWarning: invalid value encountered in sqrt
  return 2 * np.arctan( np.sqrt(A) * B )

/Users/katiechambe/Documents/Research/projects/yellowCard/yellowcard/keplerianPlane.py:53: RuntimeWarning: invalid value encountered in double_scalars
  B = ( 1 - self.e**2 )**(1/2)

/Users/katiechambe/miniconda3/envs/BADS/lib/python3.9/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)

/Users/katiechambe/miniconda3/envs/BADS/lib/python3.9/site-packages/scipy/optimize/optimize.py:2216: RuntimeWarning: invalid value encountered in double_scalars
  p = (x - v) * tmp2 - (x - w) * tmp1



In [9]:
sampler = emcee.EnsembleSampler(nwalkers = 8*len(result.x), ndim=len(result.x), log_prob_fn=model)

In [10]:
sampler_x0 = np.random.normal(result.x, 1e-2, size=(sampler.nwalkers,len(result.x)))

In [ ]:
state = sampler.run_mcmc(sampler_x0, 2000, progress=True)  # burn in
sampler.reset()
state = sampler.run_mcmc(state, 2000, progress=True)

  0%|                                                  | 0/2000 [00:00<?, ?it/s]

/Users/katiechambe/miniconda3/envs/BADS/lib/python3.9/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]



  2%|▉                                        | 44/2000 [00:43<32:19,  1.01it/s]

/Users/katiechambe/miniconda3/envs/BADS/lib/python3.9/site-packages/astropy/units/quantity.py:479: RuntimeWarning: invalid value encountered in power
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)



 67%|█████████████████████████▉             | 1332/2000 [20:56<10:09,  1.10it/s]

In [ ]:
tulips = az.from_emcee(sampler)

In [ ]:
az.plot_trace(tulips)

---

In [ ]:
last_pars = model.unpack_pars(sampler.chain[:, -1].T)
last_Lhatlg = coord.CartesianRepresentation(last_pars['Lhatlg'])
last_Lhatlg_sph = last_Lhatlg.represent_as(coord.UnitSphericalRepresentation)
plt.scatter(last_Lhatlg_sph.lon, last_Lhatlg_sph.lat)
plt.xlim(0, 2*np.pi)
plt.ylim(-np.pi, np.pi)